## Введение в нейронные сети
### Урок 4. Сверточные нейронные сети


1. Попробуйте улучшить точность распознавания образов cifar 10 сверточной нейронной сетью, рассмотренной на уроке. Приложите анализ с описанием того, что улучшает работу нейронной сети, а что ухудшает
2. Опишите в анализе, какие изменения необходимо было бы внести в получившуюся нейронную сеть, если бы ей нужно было работать не с cifar10, а с MNIST, CIFAR100 и IMAGENET

In [1]:
import scipy.io as sio

import numpy
from tensorflow.keras.datasets import cifar10
# from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import max_norm
from keras.optimizers import SGD
from keras.layers import Conv2D, MaxPooling2D
from keras import utils

import numpy as np

2023-09-25 00:44:50.446007: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-25 00:44:50.539792: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-25 00:44:50.540814: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-25 00:44:51.712549: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
seed = 7
numpy.random.seed(seed)

# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [3]:
def normalize_data(features):
    features = features.astype('float32')
    features = features / 255
    features = utils.to_categorical(features)
    return features

In [27]:
def create_model(classes_cnt, input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=input_shape, activation='relu', padding='same'))
    model.add(Dropout(0.2))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    # model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    # model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    # model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(1024, activation='relu', kernel_constraint=max_norm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu', kernel_constraint=max_norm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(classes_cnt, activation='softmax'))
    return model

In [5]:
def compile_model(model, epochs: int, learning_rate: float = 0.01):
    sgd = SGD(learning_rate=learning_rate, momentum=0.9)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [18]:
# Уменьшение выборки, а то обучение модели проходит крааайне долго
X_train = X_train[0:2000]
X_test = X_test[0:2000]
y_train = y_train[:2000]
y_test = y_test[:2000]
X_train.shape

(2000, 32, 32, 3)

In [19]:
X_train = normalize_data(X_train)
X_test = normalize_data(X_test)

In [20]:
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)

In [ ]:
model = create_model(y_test.shape[1], X_train.shape[1:])

2023-09-25 00:48:35.207407: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1610612736 exceeds 10% of free system memory.
2023-09-25 00:48:35.973547: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1610612736 exceeds 10% of free system memory.
2023-09-25 00:48:36.392013: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1610612736 exceeds 10% of free system memory.


In [1]:
model = compile_model(model, 10)
model.summary()

NameError: name 'compile_model' is not defined

In [ ]:
X_validation=X_test[0:100,:,:,:]
X_test_=X_test[100:,:,:,:]

y_validation=y_test[0:100]
y_test_=y_test[100:]

In [ ]:
model.fit(X_train, y_train, validation_data=(X_validation, y_validation), epochs=10, batch_size=32)
scores = model.evaluate(X_test_, y_test_, verbose=0)

In [ ]:
# scores
X_train[0].shape